## Yolo V3 with Tensorflow

In [1]:
from net.yolo_top import yolov3
import numpy as np
import tensorflow as tf
from data.data_pipeline import data_pipeline
from net.config import cfg
% load_ext autoreload
% autoreload 2

In [2]:
def build_data(database_path):
    # from tfrecord format database read imgs and boxes
    imgs, true_boxes = data_pipeline(database_path, cfg.batch_size)
    return imgs, true_boxes
    
def build_model(imgs, true_boxes):
    # set model status
    istraining = tf.constant(True, tf.bool)
    # build whole yolov3 model
    model = yolov3(imgs, true_boxes, istraining)
    
    # get loss
    loss = model.compute_loss()
    
    global_step = tf.Variable(0, trainable=False)
    # lr = tf.train.exponential_decay(0.0001, global_step=global_step, decay_steps=2e4, decay_rate=0.1)
    lr = tf.train.piecewise_constant(global_step, [40000, 45000], [1e-3, 1e-4, 1e-5])
    
    # get optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    # optimizer = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9)
    update_op = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    vars_det = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Head")
    # for var in vars_det:
    #     print(var)
    with tf.control_dependencies(update_op):
        train_op = optimizer.minimize(loss, global_step=global_step, var_list=vars_det)
    saver = tf.train.Saver()
    
    return model, train_op, loss, saver, global_step

def train_model(train_op, loss, saver, global_step):   
    gs = 0
    batch_per_epoch = 2000
    cfg.train.max_batches = int(batch_per_epoch * 10)
    cfg.train.image_resized = 608
    
    with tf.Session() as sess:
        ckpt = tf.train.get_checkpoint_state(cfg.path.ckpt_dir)
        if (ckpt and ckpt.model_checkpoint_path):
            saver.restore(sess, ckpt.model_checkpoint_path)
            gs = int(ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1])
            sess.run(tf.assign(global_step, gs))
            print('Restore batch: ', gs)
        else:
            print('no checkpoint found')
            sess.run(tf.global_variables_initializer())
        for i in range(gs, cfg.train.max_batches):
            _, loss_ = sess.run([train_op, loss])
            if(i % 100 == 0):
                print(i,': ', loss_)
            if(i % 1000 == 0):
                saver.save(sess, cfg.path.ckpt_dir+'yolov3.ckpt', global_step=global_step, write_meta_graph=False)

In [3]:
imgs, true_boxes = build_data(cfg.path.train_data_path)
model, train_op, loss, saver, global_step = build_model(imgs, true_boxes)

darknet
(?, 608, 608, 3)


/home/tsimage-x/anaconda3/envs/tensorflow-work/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [4]:
print(imgs.shape)
print(model.img.shape)

(?, 608, 608, 3)
(?, 608, 608, 3)


In [ ]:
train_model(train_op, loss, saver, global_step)

INFO:tensorflow:Restoring parameters from ./ckpt/yolov3.ckpt-1
Restore batch:  1
100 :  185.073
200 :  321.729
300 :  75.4954
400 :  113.673
500 :  75.3663
600 :  85.4102
700 :  54.4702
800 :  90.2194
900 :  43.5213
1000 :  43.3888
1100 :  78.0176
1200 :  46.2254
1300 :  56.5917
1400 :  106.116
1500 :  63.1553
1600 :  49.5945
1700 :  59.0254
1800 :  43.4884
1900 :  47.833
2000 :  57.9187
2100 :  111.791
2200 :  65.7887
2300 :  45.7487
2400 :  43.7859
2500 :  37.265
2600 :  39.9885
2700 :  51.0216
2800 :  44.2853
2900 :  32.4721
3000 :  33.8502
3100 :  31.3938
3200 :  38.6412
3300 :  37.0735
3400 :  45.5054
3500 :  32.663
3600 :  51.0515
3700 :  60.4868
3800 :  66.2721
3900 :  24.2105
4000 :  44.0316
4100 :  34.9523
4200 :  34.3413
4300 :  40.8378
4400 :  31.5358
4500 :  69.7347
4600 :  49.0802
4700 :  29.9731
4800 :  31.1307
4900 :  31.727
5000 :  61.2346
5100 :  27.8853
5200 :  24.304
5300 :  19.6182
5400 :  33.8224
5500 :  36.1826
5600 :  22.9399
5700 :  35.7016
5800 :  28.2625
5900 